<a href="https://colab.research.google.com/github/santossousa/Machine-learning-models/blob/main/predict_price_close_bitcoin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introdução

A previsão de preços é uma tarefa desafiadora em muitos mercados, e o mercado de criptomoedas não é uma exceção. Com a volatilidade e complexidade dos preços das criptomoedas, ter a capacidade de prever com precisão os movimentos futuros dos preços pode ser extremamente valioso para investidores e traders.


Nesse contexto, as redes neurais LSTM (Long Short-Term Memory) têm se mostrado uma abordagem eficaz para a previsão de séries temporais, incluindo a previsão de preços de criptomoedas. As LSTMs são um tipo especial de rede neural recorrente que possui uma memória de longo prazo, permitindo capturar dependências de longo prazo nos dados. 


Neste contexto, este projeto tem como objetivo desenvolver um modelo de previsão de preços usando uma rede neural LSTM para o BTC-USD (Bitcoin em relação ao Dólar Americano). Serão utilizados dados históricos de preços do BTC-USD para treinar o modelo e fazer previsões futuras.


O modelo será construído usando a biblioteca Keras, que fornece uma interface simples e poderosa para criar redes neurais usando Python. Será utilizada a arquitetura de rede LSTM, utilizando camadas LSTM e camadas densas para realizar a previsão do preço de fechamento.



Ao final do projeto, teremos um modelo treinado capaz de prever preços futuros de fechamento do BTC-USD com base em padrões e tendências extraídos de dados históricos. Essas previsões podem ser usadas para auxiliar na tomada de decisões de investimento ou no desenvolvimento de estratégias de negociação.


In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.express as px 
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from datetime import datetime

In [2]:
class PredictBiticionCloseFuture:

  def __init__(self):

    self.__symbol = "BTC-USD"
    self.__btc_data = yf.download(self.__symbol, start="2022-01-01", end=datetime.today().strftime("%Y-%m-%d"))
    self.__df = pd.DataFrame(self.__btc_data)
    self.__df = self.__df.reset_index()
    self.__scaler = MinMaxScaler(feature_range=(0, 1))
    self.__look_back = 0
    self.__num_predictions = 0

  def __close_bitcoin(self):

    return self.__df['Close'].values.reshape(-1, 1)

  def __normaliz_values(self):

    return  self.__scaler.fit_transform(self.__close_bitcoin())

  def train_test(self, days=180):

    look_back = days
    self.__look_back = look_back

    normalized_values = self.__normaliz_values()

    X = np.array([normalized_values[i-look_back:i] for i in range(look_back, len(normalized_values))])
    y = np.array([normalized_values[i] for i in range(look_back, len(normalized_values))])

    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]


    return X_train, y_train, X_test, y_test


  def __model(self):

    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(self.__look_back, 1)))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

  def __fit_data(self):

    model = self.__model()
    X_train, y_train, _, _ = self.train_test()
    model.fit(X_train, y_train, epochs=10, batch_size=32)

    return model
  
  def predict(self, num_predictions=67):

    if num_predictions > 67:

      raise Exception("Limite de previsão ultrapassado!!")
      
    _,_, X_test, y_test = self.train_test()
    self.__num_predictions = min(len(X_test), num_predictions)  # Número de dias de previsão
    model = self.__fit_data()
    
    predictions = model.predict(X_test[:num_predictions])
    predicted_values = self.__scaler.inverse_transform(predictions)

    actual_values = self.__scaler.inverse_transform(y_test[:self.__num_predictions])


    return predicted_values, actual_values

  def close_future_months(self):

    predicted_values, actual_values = self.predict()

    last_date = pd.to_datetime(self.__df['Date'].iloc[-1])
    next_dates = pd.date_range(start=last_date + pd.DateOffset(days=1), periods=self.__num_predictions+1, inclusive='right')
    
    std_dev = np.std(predicted_values)
    confidence_interval = 1.96 * std_dev 
    
    lower_bound = predicted_values - confidence_interval
    upper_bound = predicted_values + confidence_interval
    
    error_percentage = (abs(predicted_values - actual_values) / actual_values) * 100
    
    df_predictions = pd.DataFrame({'Date': next_dates,\
                                   'Predicted Close': predicted_values.flatten(),\
                                   'Low Predicted Close':lower_bound.flatten(),\
                                   'Upper Predicted Close':upper_bound.flatten(),\
                                   'Absolute Error':error_percentage.flatten()})
    
    return df_predictions
  
  def trend(self):
  
    fig = px.line(self.__df, x='Date', y='Close', title='Preço de fechamento do Bitcoin')

    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
                ])
            )
        )
    fig.show()

  def graph_predicitons(self):

    df_predictions = self.close_future_months()

    fig = px.line(df_predictions, x='Date', y='Predicted Close', title='Previsão dos próximos mês')
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=2, label="2m", step="month", stepmode="backward"),
                dict(step="all")
                ])
            )
        )
    fig.show()

  def graph_bar(self):
    df = self.get_best_days(5,'close')
    fig = px.bar(df, x='Date', y='Predicted Close', color='Month')
    fig.show()
    df_ = self.get_best_days(5,'error')
    fig = px.bar(df_, x='Date', y='Absolute Error', color='Month')
    fig.show()
    
  def get_best_days(self, num_days, criteria):

    df_predictions = self.close_future_months()

    df_predictions['Month'] = df_predictions['Date'].dt.month
    df_predictions['Year'] = df_predictions['Date'].dt.year
    
    next_two_months = df_predictions[df_predictions['Date'].dt.month <= (df_predictions['Month'].max() + 2)]
    
    if criteria == 'close':

        best_days = next_two_months.groupby(['Year', 'Month']).apply(lambda x: x.nlargest(num_days, 'Predicted Close')).reset_index(drop=True)

    elif criteria == 'error':

        best_days = next_two_months.groupby(['Year', 'Month']).apply(lambda x: x.nsmallest(num_days, 'Absolute Error')).reset_index(drop=True)

    else:

        raise ValueError("Valor não suportado!!")
    
    return best_days


In [3]:
predict_biticion = PredictBiticionCloseFuture()

[*********************100%***********************]  1 of 1 completed


In [4]:
predict_biticion.trend()

In [5]:
predict_biticion.close_future_months()

Epoch 1/10
9/9 [==============================] - 4s 137ms/step - loss: 0.0104
Epoch 2/10
9/9 [==============================] - 1s 107ms/step - loss: 0.0038
Epoch 3/10
9/9 [==============================] - 1s 105ms/step - loss: 0.0030
Epoch 4/10
9/9 [==============================] - 1s 104ms/step - loss: 0.0025
Epoch 5/10
9/9 [==============================] - 1s 105ms/step - loss: 0.0023
Epoch 6/10
9/9 [==============================] - 1s 106ms/step - loss: 0.0020
Epoch 7/10
9/9 [==============================] - 1s 107ms/step - loss: 0.0022
Epoch 8/10
9/9 [==============================] - 1s 107ms/step - loss: 0.0021
Epoch 9/10
9/9 [==============================] - 1s 106ms/step - loss: 0.0019
Epoch 10/10
3/3 [==============================] - 1s 21ms/step


,Date,Predicted Close,Low Predicted Close,Upper Predicted Close,Absolute Error
0,2023-06-02,26949.654297,25257.703125,28641.605469,1.982391
1,2023-06-03,27233.927734,25541.976562,28925.878906,2.716273
2,2023-06-04,27467.820312,25775.869141,29159.771484,1.208301
3,2023-06-05,27594.906250,25902.955078,29286.857422,1.198378
4,2023-06-06,27652.904297,25960.953125,29344.855469,2.453529
...,...,...,...,...,...
62,2023-08-03,26423.648438,24731.697266,28115.599609,1.106476
63,2023-08-04,26384.673828,24692.722656,28076.625000,1.800184
64,2023-08-05,26366.035156,24674.083984,28057.986328,6.122741
65,2023-08-06,26441.134766,24749.183594,28133.085938,4.702499


In [6]:
predict_biticion.graph_predicitons()

Epoch 1/10
9/9 [==============================] - 3s 107ms/step - loss: 0.0130
Epoch 2/10
9/9 [==============================] - 1s 156ms/step - loss: 0.0047
Epoch 3/10
9/9 [==============================] - 1s 122ms/step - loss: 0.0032
Epoch 4/10
9/9 [==============================] - 1s 107ms/step - loss: 0.0025
Epoch 5/10
9/9 [==============================] - 1s 107ms/step - loss: 0.0020
Epoch 6/10
9/9 [==============================] - 1s 107ms/step - loss: 0.0019
Epoch 7/10
9/9 [==============================] - 1s 108ms/step - loss: 0.0018
Epoch 8/10
9/9 [==============================] - 1s 108ms/step - loss: 0.0018
Epoch 9/10
9/9 [==============================] - 1s 105ms/step - loss: 0.0017
Epoch 10/10
3/3 [==============================] - 1s 26ms/step


In [7]:
predict_biticion.get_best_days(5,'close')

Epoch 1/10
9/9 [==============================] - 4s 111ms/step - loss: 0.0145
Epoch 2/10
9/9 [==============================] - 1s 109ms/step - loss: 0.0049
Epoch 3/10
9/9 [==============================] - 1s 107ms/step - loss: 0.0034
Epoch 4/10
9/9 [==============================] - 1s 107ms/step - loss: 0.0027
Epoch 5/10
9/9 [==============================] - 1s 106ms/step - loss: 0.0022
Epoch 6/10
9/9 [==============================] - 1s 107ms/step - loss: 0.0019
Epoch 7/10
9/9 [==============================] - 1s 108ms/step - loss: 0.0018
Epoch 8/10
9/9 [==============================] - 1s 132ms/step - loss: 0.0017
Epoch 9/10
9/9 [==============================] - 1s 142ms/step - loss: 0.0017
Epoch 10/10
3/3 [==============================] - 1s 22ms/step


,Date,Predicted Close,Low Predicted Close,Upper Predicted Close,Absolute Error,Month,Year
0,2023-06-27,29494.765625,27815.765625,31173.765625,2.331795,6,2023
1,2023-06-28,29480.050781,27801.050781,31159.050781,4.368983,6,2023
2,2023-06-26,29379.142578,27700.142578,31058.142578,3.350303,6,2023
3,2023-06-29,29347.968750,27668.968750,31026.968750,7.592717,6,2023
4,2023-06-25,29264.236328,27585.236328,30943.236328,0.614054,6,2023
5,2023-07-01,28785.970703,27106.970703,30464.970703,4.329561,7,2023
6,2023-07-02,28474.763672,26795.763672,30153.763672,3.449272,7,2023
7,2023-07-16,28321.806641,26642.806641,30000.806641,2.265931,7,2023
8,2023-07-15,28314.189453,26635.189453,29993.189453,0.494778,7,2023
9,2023-07-14,28256.849609,26577.849609,29935.849609,2.241072,7,2023


In [8]:
predict_biticion.get_best_days(5,'error')

Epoch 1/10
9/9 [==============================] - 4s 114ms/step - loss: 0.0102
Epoch 2/10
9/9 [==============================] - 1s 113ms/step - loss: 0.0033
Epoch 3/10
9/9 [==============================] - 1s 113ms/step - loss: 0.0028
Epoch 4/10
9/9 [==============================] - 1s 109ms/step - loss: 0.0024
Epoch 5/10
9/9 [==============================] - 1s 113ms/step - loss: 0.0022
Epoch 6/10
9/9 [==============================] - 1s 125ms/step - loss: 0.0020
Epoch 7/10
9/9 [==============================] - 1s 161ms/step - loss: 0.0020
Epoch 8/10
9/9 [==============================] - 1s 113ms/step - loss: 0.0019
Epoch 9/10
9/9 [==============================] - 1s 115ms/step - loss: 0.0018
Epoch 10/10
3/3 [==============================] - 1s 24ms/step


,Date,Predicted Close,Low Predicted Close,Upper Predicted Close,Absolute Error,Month,Year
0,2023-06-04,27069.355469,25439.777344,28698.933594,0.259888,6,2023
1,2023-06-05,27165.800781,25536.222656,28795.378906,0.375274,6,2023
2,2023-06-11,27528.042969,25898.464844,29157.621094,0.943417,6,2023
3,2023-06-25,29072.783203,27443.205078,30702.361328,1.264259,6,2023
4,2023-06-15,27569.396484,25939.818359,29198.974609,1.276462,6,2023
5,2023-07-09,27970.820312,26341.242188,29600.398438,0.429837,7,2023
6,2023-07-23,27048.052734,25418.474609,28677.630859,0.531910,7,2023
7,2023-07-24,26883.759766,25254.181641,28513.337891,0.565494,7,2023
8,2023-07-26,26665.746094,25036.167969,28295.324219,0.620385,7,2023
9,2023-07-03,28101.726562,26472.148438,29731.304688,0.727264,7,2023
